In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# 读取年数据和日数据
#load data
starttime = datetime.datetime.now()
#读取年数据
data_year = pd.read_csv('Data/年数据.csv',encoding = 'GBK',engine='python')

#读取日数据
data_day = pd.read_csv('Data/日数据.csv',encoding = 'GBK',engine='python')

endtime = datetime.datetime.now()

print('数据读取用时：%.2f秒'%(endtime - starttime).seconds)

数据读取用时：713.00秒


In [9]:
data_day.shape,data_year.shape

((5899132, 61), (24262, 362))

In [3]:
# 关键字，股票编号和年份
keys = data_year[['股票编号','年份（年末）']]

In [10]:
# 日数据列名信息
data_day.columns

Index(['股票编号', '年', '月', '日', '开盘价', '最高价', '最低价', '收盘价', '成交量', '成交金额',
       '息前税后利润', '息税前利润', '企业自由现金流量', '毛利', '无息流动负债', '无息非流动负债', '净债务',
       '净营运资本', '留存收益', '全部投入资本', '120日收益损失方差比', '120日收益方差', '个股收益的120日峰度',
       '120日损失方差', '120日方差', '120日信息比率(InformationRatio120)', '个股120日的beta值',
       '个股120日的alpha', '120日特诺雷比率', '120日夏普比率', '波幅中位数', '资金流量指标', '过去1年的价格动量',
       '月度换手率对数', '120日平均换手率', '当前价格处于过去1年股价的位置', '流通市值', '市净率', '市盈率',
       '过去3个月的价格动量', '基本每股收益', '对数流通市值', '总市值', '市现率', '市销率', '稀释每股收益',
       '每股收益TTM值', '每股净资产', '每股营业收入', '每股营业总收入', '每股资本公积金', '每股息税前利润',
       '每股营业利润', '每股盈余公积金', '每股未分配利润', '每股现金流量净额', '每股股利', '每股企业自由现金流量',
       '每股经营活动产生的现金流量净额', '每股留存收益', '每股股东自由现金流量'],
      dtype='object')

In [11]:
# 对于每个（股票，年份）数据对，取其年末20天的收盘价根据成交量为权重取加权平均
def get_last_avg(code,year):
    info = data_day[data_day['股票编号'] == code]
    info = info[info['年'] == year]
    if len(info) >= 20:
        price_list = info['收盘价'].values[-20:]
        volume_list = info['成交量'].values[-20:]
        if np.sum(volume_list) > 0:
            avg_price = np.sum([price_list[i] * volume_list[i] for i in range(len(price_list))])/np.sum(volume_list)
        else:
            avg_price = price_list[-1]
        return avg_price
    elif len(info) > 0:
        price_list = info['收盘价'].values
        volume_list = info['成交量'].values
        if np.sum(volume_list) > 0:
            avg_price = np.sum([price_list[i] * volume_list[i] for i in range(len(price_list))])/np.sum(volume_list)
        else:
            # 年末最后20个交易日成交量都为0的情况
            avg_price = price_list[-1]
        return avg_price
    else:
        return None

# 对于每个（股票，年份）数据对，计算年波动率
#年股票波动率：对数收益率的标准差再乘有效交易日个数的平方根
def get_vol(code,year):
    info = data_day[data_day['股票编号'] == code]
    info = info[info['年'] == year]
    # 计算该年份收盘价对数收益率的序列
    logreturns = np.diff(np.log(info['收盘价']))
    # 对数收益率的标准差，乘有效交易日天数的平方根，为股票年化波动率
    annualVolatility = np.std(logreturns) * np.sqrt(len(info))
    return annualVolatility

In [12]:
print('例：股票编号%d，第%d年份年末最后20个交易日收盘价加权平均值为:%.4f'%(1,5,get_last_avg(1,5)))
print('例：股票编号%d，第%d年份年的年化波动率为为:%.4f'%(1054,5,get_vol(1054,5)))

例：股票编号1，第5年份年末最后20个交易日收盘价加权平均值为:13.9475
例：股票编号1054，第5年份年的年化波动率为为:0.6812


In [13]:
# 获取有效记录的数据对（股票，年份）的平均股价和波动率因子

price_lists = []
vol_lists = []
for code,year in zip(keys['股票编号'].values,keys['年份（年末）'].values):
    avg_price = get_last_avg(code,year)
    price_lists.append(avg_price)
    
    vol = get_vol(code,year)
    vol_lists.append(vol)
    print('股票编号%d，第%d年份年末最后20个交易日收盘价加权平均值为:%.4f，年化波动率为：%.2f'%(code,year,avg_price,vol))

股票编号1，第1年份年末最后20个交易日收盘价加权平均值为:6.1864，年化波动率为：0.50
股票编号1，第2年份年末最后20个交易日收盘价加权平均值为:5.8128，年化波动率为：0.49
股票编号1，第3年份年末最后20个交易日收盘价加权平均值为:7.1599，年化波动率为：0.42
股票编号1，第4年份年末最后20个交易日收盘价加权平均值为:9.0881，年化波动率为：0.34
股票编号1，第5年份年末最后20个交易日收盘价加权平均值为:13.9475，年化波动率为：0.75
股票编号1，第6年份年末最后20个交易日收盘价加权平均值为:17.7460，年化波动率为：0.56
股票编号1，第7年份年末最后20个交易日收盘价加权平均值为:15.9563，年化波动率为：0.39
股票编号2，第1年份年末最后20个交易日收盘价加权平均值为:8.3902，年化波动率为：0.00
股票编号2，第2年份年末最后20个交易日收盘价加权平均值为:8.3902，年化波动率为：0.00
股票编号2，第3年份年末最后20个交易日收盘价加权平均值为:8.3902，年化波动率为：0.00
股票编号2，第4年份年末最后20个交易日收盘价加权平均值为:8.3902，年化波动率为：0.00
股票编号2，第5年份年末最后20个交易日收盘价加权平均值为:8.3902，年化波动率为：0.00
股票编号2，第6年份年末最后20个交易日收盘价加权平均值为:8.3902，年化波动率为：0.00
股票编号2，第7年份年末最后20个交易日收盘价加权平均值为:17.2456，年化波动率为：0.47
股票编号3，第1年份年末最后20个交易日收盘价加权平均值为:2.0900，年化波动率为：0.34
股票编号3，第2年份年末最后20个交易日收盘价加权平均值为:2.6003，年化波动率为：0.47
股票编号3，第3年份年末最后20个交易日收盘价加权平均值为:2.0901，年化波动率为：0.28
股票编号3，第4年份年末最后20个交易日收盘价加权平均值为:3.5909，年化波动率为：0.34
股票编号3，第5年份年末最后20个交易日收盘价加权平均值为:5.1832，年化波动率为：0.71
股票编号3，第6年份年末最后20个交易日收盘价加权平均值为:7.4186，年化波动率为：0.50
股票编号3，第7年份年末最后20

股票编号25，第1年份年末最后20个交易日收盘价加权平均值为:12.6808，年化波动率为：0.00
股票编号25，第2年份年末最后20个交易日收盘价加权平均值为:12.6808，年化波动率为：0.00
股票编号25，第3年份年末最后20个交易日收盘价加权平均值为:12.6808，年化波动率为：0.00
股票编号25，第4年份年末最后20个交易日收盘价加权平均值为:12.6808，年化波动率为：0.00
股票编号25，第5年份年末最后20个交易日收盘价加权平均值为:12.6808，年化波动率为：0.00
股票编号25，第6年份年末最后20个交易日收盘价加权平均值为:12.6808，年化波动率为：0.00
股票编号25，第7年份年末最后20个交易日收盘价加权平均值为:11.4155，年化波动率为：0.42
股票编号26，第1年份年末最后20个交易日收盘价加权平均值为:2.9072，年化波动率为：0.31
股票编号26，第2年份年末最后20个交易日收盘价加权平均值为:2.4178，年化波动率为：0.22
股票编号26，第3年份年末最后20个交易日收盘价加权平均值为:2.0257，年化波动率为：0.33
股票编号26，第4年份年末最后20个交易日收盘价加权平均值为:3.7348，年化波动率为：0.31
股票编号26，第5年份年末最后20个交易日收盘价加权平均值为:4.3071，年化波动率为：0.66
股票编号26，第6年份年末最后20个交易日收盘价加权平均值为:2.9972，年化波动率为：0.30
股票编号26，第7年份年末最后20个交易日收盘价加权平均值为:2.8227，年化波动率为：0.19
股票编号27，第1年份年末最后20个交易日收盘价加权平均值为:5.5324，年化波动率为：0.00
股票编号27，第2年份年末最后20个交易日收盘价加权平均值为:5.5324，年化波动率为：0.00
股票编号27，第3年份年末最后20个交易日收盘价加权平均值为:5.5324，年化波动率为：0.00
股票编号27，第4年份年末最后20个交易日收盘价加权平均值为:5.5324，年化波动率为：0.00
股票编号27，第5年份年末最后20个交易日收盘价加权平均值为:5.5324，年化波动率为：0.00
股票编号27，第6年份年末最后20个交易日收盘价加权平均值为:36.4983，年化波动

股票编号48，第7年份年末最后20个交易日收盘价加权平均值为:6.0150，年化波动率为：0.28
股票编号49，第1年份年末最后20个交易日收盘价加权平均值为:3.1517，年化波动率为：0.40
股票编号49，第2年份年末最后20个交易日收盘价加权平均值为:3.1924，年化波动率为：0.37
股票编号49，第3年份年末最后20个交易日收盘价加权平均值为:3.8705，年化波动率为：0.31
股票编号49，第4年份年末最后20个交易日收盘价加权平均值为:14.7860，年化波动率为：0.49
股票编号49，第5年份年末最后20个交易日收盘价加权平均值为:19.6760，年化波动率为：0.81
股票编号49，第6年份年末最后20个交易日收盘价加权平均值为:15.3276，年化波动率为：0.35
股票编号49，第7年份年末最后20个交易日收盘价加权平均值为:10.8300，年化波动率为：0.35
股票编号50，第1年份年末最后20个交易日收盘价加权平均值为:5.3323，年化波动率为：0.00
股票编号50，第2年份年末最后20个交易日收盘价加权平均值为:4.9162，年化波动率为：0.46
股票编号50，第3年份年末最后20个交易日收盘价加权平均值为:12.3516，年化波动率为：0.54
股票编号50，第4年份年末最后20个交易日收盘价加权平均值为:14.8289，年化波动率为：0.57
股票编号50，第5年份年末最后20个交易日收盘价加权平均值为:21.3953，年化波动率为：0.80
股票编号50，第6年份年末最后20个交易日收盘价加权平均值为:18.5461，年化波动率为：0.52
股票编号50，第7年份年末最后20个交易日收盘价加权平均值为:11.5287，年化波动率为：0.28
股票编号51，第1年份年末最后20个交易日收盘价加权平均值为:13.2388，年化波动率为：0.35
股票编号51，第2年份年末最后20个交易日收盘价加权平均值为:9.0966，年化波动率为：0.43
股票编号51，第3年份年末最后20个交易日收盘价加权平均值为:9.5913，年化波动率为：0.49
股票编号51，第4年份年末最后20个交易日收盘价加权平均值为:11.5063，年化波动率为：0.36
股票编号51，第5年份年末最后20个交易日收盘价加权平均值为:13.4411，

股票编号72，第7年份年末最后20个交易日收盘价加权平均值为:18.4284，年化波动率为：0.34
股票编号73，第1年份年末最后20个交易日收盘价加权平均值为:1.1383，年化波动率为：0.28
股票编号73，第2年份年末最后20个交易日收盘价加权平均值为:1.0676，年化波动率为：0.23
股票编号73，第3年份年末最后20个交易日收盘价加权平均值为:1.0591，年化波动率为：0.29
股票编号73，第4年份年末最后20个交易日收盘价加权平均值为:5.1154，年化波动率为：0.56
股票编号73，第5年份年末最后20个交易日收盘价加权平均值为:4.5654，年化波动率为：0.61
股票编号73，第6年份年末最后20个交易日收盘价加权平均值为:3.3941，年化波动率为：0.33
股票编号73，第7年份年末最后20个交易日收盘价加权平均值为:3.3219，年化波动率为：0.24
股票编号74，第1年份年末最后20个交易日收盘价加权平均值为:3.0711，年化波动率为：0.47
股票编号74，第2年份年末最后20个交易日收盘价加权平均值为:2.0846，年化波动率为：0.48
股票编号74，第3年份年末最后20个交易日收盘价加权平均值为:2.6400，年化波动率为：0.37
股票编号74，第4年份年末最后20个交易日收盘价加权平均值为:4.2866，年化波动率为：0.39
股票编号74，第5年份年末最后20个交易日收盘价加权平均值为:4.9395，年化波动率为：0.69
股票编号74，第6年份年末最后20个交易日收盘价加权平均值为:3.3700，年化波动率为：0.50
股票编号74，第7年份年末最后20个交易日收盘价加权平均值为:2.4267，年化波动率为：0.19
股票编号75，第1年份年末最后20个交易日收盘价加权平均值为:4.7090，年化波动率为：0.00
股票编号75，第2年份年末最后20个交易日收盘价加权平均值为:4.7090，年化波动率为：0.00
股票编号75，第3年份年末最后20个交易日收盘价加权平均值为:4.7090，年化波动率为：0.00
股票编号75，第4年份年末最后20个交易日收盘价加权平均值为:4.7090，年化波动率为：0.00
股票编号75，第5年份年末最后20个交易日收盘价加权平均值为:28.2634，年化波动率为：0.8

股票编号96，第7年份年末最后20个交易日收盘价加权平均值为:3.9340，年化波动率为：0.34
股票编号97，第1年份年末最后20个交易日收盘价加权平均值为:3.5713，年化波动率为：0.47
股票编号97，第2年份年末最后20个交易日收盘价加权平均值为:2.9325，年化波动率为：0.35
股票编号97，第3年份年末最后20个交易日收盘价加权平均值为:10.0924，年化波动率为：0.53
股票编号97，第4年份年末最后20个交易日收盘价加权平均值为:17.6925，年化波动率为：0.45
股票编号97，第5年份年末最后20个交易日收盘价加权平均值为:29.3914，年化波动率为：0.76
股票编号97，第6年份年末最后20个交易日收盘价加权平均值为:19.4482，年化波动率为：0.42
股票编号97，第7年份年末最后20个交易日收盘价加权平均值为:19.2731，年化波动率为：0.35
股票编号98，第1年份年末最后20个交易日收盘价加权平均值为:11.9557，年化波动率为：0.00
股票编号98，第2年份年末最后20个交易日收盘价加权平均值为:11.9557，年化波动率为：0.00
股票编号98，第3年份年末最后20个交易日收盘价加权平均值为:11.9557，年化波动率为：0.00
股票编号98，第4年份年末最后20个交易日收盘价加权平均值为:11.9557，年化波动率为：0.00
股票编号98，第5年份年末最后20个交易日收盘价加权平均值为:11.9557，年化波动率为：0.00
股票编号98，第6年份年末最后20个交易日收盘价加权平均值为:33.0078，年化波动率为：0.59
股票编号98，第7年份年末最后20个交易日收盘价加权平均值为:17.0998，年化波动率为：0.39
股票编号99，第1年份年末最后20个交易日收盘价加权平均值为:5.7736，年化波动率为：0.38
股票编号99，第2年份年末最后20个交易日收盘价加权平均值为:5.3611，年化波动率为：0.43
股票编号99，第3年份年末最后20个交易日收盘价加权平均值为:10.2521，年化波动率为：0.43
股票编号99，第4年份年末最后20个交易日收盘价加权平均值为:13.2319，年化波动率为：0.47
股票编号99，第5年份年末最后20个交易日收盘价加权平均值为:24.08

...

股票编号3368，第2年份年末最后20个交易日收盘价加权平均值为:1.6478，年化波动率为：0.36
股票编号3368，第3年份年末最后20个交易日收盘价加权平均值为:2.0108，年化波动率为：0.34
股票编号3368，第4年份年末最后20个交易日收盘价加权平均值为:3.0419，年化波动率为：0.37
股票编号3368，第5年份年末最后20个交易日收盘价加权平均值为:11.3795，年化波动率为：0.75
股票编号3368，第6年份年末最后20个交易日收盘价加权平均值为:7.2468，年化波动率为：0.42
股票编号3368，第7年份年末最后20个交易日收盘价加权平均值为:5.8005，年化波动率为：0.23
股票编号3369，第1年份年末最后20个交易日收盘价加权平均值为:6.4608，年化波动率为：0.00
股票编号3369，第2年份年末最后20个交易日收盘价加权平均值为:6.4608，年化波动率为：0.00
股票编号3369，第3年份年末最后20个交易日收盘价加权平均值为:6.4608，年化波动率为：0.00
股票编号3369，第4年份年末最后20个交易日收盘价加权平均值为:6.4608，年化波动率为：0.00
股票编号3369，第5年份年末最后20个交易日收盘价加权平均值为:6.4608，年化波动率为：0.00
股票编号3369，第6年份年末最后20个交易日收盘价加权平均值为:24.7735，年化波动率为：0.66
股票编号3369，第7年份年末最后20个交易日收盘价加权平均值为:13.8854，年化波动率为：0.38
股票编号3370，第1年份年末最后20个交易日收盘价加权平均值为:7.7056，年化波动率为：0.39
股票编号3370，第2年份年末最后20个交易日收盘价加权平均值为:6.3229，年化波动率为：0.41
股票编号3370，第3年份年末最后20个交易日收盘价加权平均值为:4.5651，年化波动率为：0.31
股票编号3370，第4年份年末最后20个交易日收盘价加权平均值为:7.5677，年化波动率为：0.35
股票编号3370，第5年份年末最后20个交易日收盘价加权平均值为:5.9846，年化波动率为：0.64
股票编号3370，第6年份年末最后20个交易日收盘价加权平均值为:5.4403，年化波动率为：0.35
股票编号3370，

股票编号3392，第4年份年末最后20个交易日收盘价加权平均值为:4.0766，年化波动率为：0.39
股票编号3392，第5年份年末最后20个交易日收盘价加权平均值为:4.0063，年化波动率为：0.48
股票编号3392，第6年份年末最后20个交易日收盘价加权平均值为:10.3574，年化波动率为：0.50
股票编号3392，第7年份年末最后20个交易日收盘价加权平均值为:6.5785，年化波动率为：0.37
股票编号3393，第1年份年末最后20个交易日收盘价加权平均值为:1.1922，年化波动率为：0.35
股票编号3393，第2年份年末最后20个交易日收盘价加权平均值为:2.1133，年化波动率为：0.43
股票编号3393，第3年份年末最后20个交易日收盘价加权平均值为:2.2528，年化波动率为：0.38
股票编号3393，第4年份年末最后20个交易日收盘价加权平均值为:3.5643，年化波动率为：0.33
股票编号3393，第5年份年末最后20个交易日收盘价加权平均值为:4.4477，年化波动率为：0.64
股票编号3393，第6年份年末最后20个交易日收盘价加权平均值为:3.5586，年化波动率为：0.33
股票编号3393，第7年份年末最后20个交易日收盘价加权平均值为:3.4918，年化波动率为：0.14
股票编号3394，第1年份年末最后20个交易日收盘价加权平均值为:17.2320，年化波动率为：0.41
股票编号3394，第2年份年末最后20个交易日收盘价加权平均值为:13.7220，年化波动率为：0.33
股票编号3394，第3年份年末最后20个交易日收盘价加权平均值为:7.7519，年化波动率为：0.40
股票编号3394，第4年份年末最后20个交易日收盘价加权平均值为:10.5107，年化波动率为：0.42
股票编号3394，第5年份年末最后20个交易日收盘价加权平均值为:7.9628，年化波动率为：0.69
股票编号3394，第6年份年末最后20个交易日收盘价加权平均值为:9.1501，年化波动率为：0.51
股票编号3394，第7年份年末最后20个交易日收盘价加权平均值为:11.6243，年化波动率为：0.35
股票编号3395，第1年份年末最后20个交易日收盘价加权平均值为:0.7836，年化波动率为：0.38
股票编号339

股票编号3416，第3年份年末最后20个交易日收盘价加权平均值为:4.5448，年化波动率为：0.40
股票编号3416，第4年份年末最后20个交易日收盘价加权平均值为:6.4356，年化波动率为：0.38
股票编号3416，第5年份年末最后20个交易日收盘价加权平均值为:11.8661，年化波动率为：0.76
股票编号3416，第6年份年末最后20个交易日收盘价加权平均值为:16.2749，年化波动率为：0.52
股票编号3416，第7年份年末最后20个交易日收盘价加权平均值为:11.0405，年化波动率为：0.42
股票编号3417，第1年份年末最后20个交易日收盘价加权平均值为:9.8164，年化波动率为：0.46
股票编号3417，第2年份年末最后20个交易日收盘价加权平均值为:5.3122，年化波动率为：0.37
股票编号3417，第3年份年末最后20个交易日收盘价加权平均值为:6.1279，年化波动率为：0.37
股票编号3417，第4年份年末最后20个交易日收盘价加权平均值为:9.0993，年化波动率为：0.37
股票编号3417，第5年份年末最后20个交易日收盘价加权平均值为:7.5768，年化波动率为：0.70
股票编号3417，第6年份年末最后20个交易日收盘价加权平均值为:8.1462，年化波动率为：0.39
股票编号3417，第7年份年末最后20个交易日收盘价加权平均值为:5.6178，年化波动率为：0.31
股票编号3418，第1年份年末最后20个交易日收盘价加权平均值为:9.6958，年化波动率为：0.00
股票编号3418，第2年份年末最后20个交易日收盘价加权平均值为:9.6958，年化波动率为：0.00
股票编号3418，第3年份年末最后20个交易日收盘价加权平均值为:9.6958，年化波动率为：0.00
股票编号3418，第4年份年末最后20个交易日收盘价加权平均值为:9.6958，年化波动率为：0.00
股票编号3418，第5年份年末最后20个交易日收盘价加权平均值为:9.6958，年化波动率为：0.00
股票编号3418，第6年份年末最后20个交易日收盘价加权平均值为:9.6958，年化波动率为：0.00
股票编号3418，第7年份年末最后20个交易日收盘价加权平均值为:13.9009，年化波动率为：0.43
股票编号3419

股票编号3440，第3年份年末最后20个交易日收盘价加权平均值为:3.7651，年化波动率为：0.47
股票编号3440，第4年份年末最后20个交易日收盘价加权平均值为:9.4912，年化波动率为：0.59
股票编号3440，第5年份年末最后20个交易日收盘价加权平均值为:17.5414，年化波动率为：0.46
股票编号3440，第6年份年末最后20个交易日收盘价加权平均值为:15.6705，年化波动率为：0.54
股票编号3440，第7年份年末最后20个交易日收盘价加权平均值为:12.2898，年化波动率为：0.41
股票编号3441，第1年份年末最后20个交易日收盘价加权平均值为:7.9152，年化波动率为：0.35
股票编号3441，第2年份年末最后20个交易日收盘价加权平均值为:9.7479，年化波动率为：0.37
股票编号3441，第3年份年末最后20个交易日收盘价加权平均值为:18.9165，年化波动率为：0.47
股票编号3441，第4年份年末最后20个交易日收盘价加权平均值为:22.3762，年化波动率为：0.37
股票编号3441，第5年份年末最后20个交易日收盘价加权平均值为:26.7371，年化波动率为：0.63
股票编号3441，第6年份年末最后20个交易日收盘价加权平均值为:35.0274，年化波动率为：0.40
股票编号3441，第7年份年末最后20个交易日收盘价加权平均值为:46.0686，年化波动率为：0.26
股票编号3442，第1年份年末最后20个交易日收盘价加权平均值为:3.6554，年化波动率为：0.45
股票编号3442，第2年份年末最后20个交易日收盘价加权平均值为:3.4556，年化波动率为：0.43
股票编号3442，第3年份年末最后20个交易日收盘价加权平均值为:3.2371，年化波动率为：0.42
股票编号3442，第4年份年末最后20个交易日收盘价加权平均值为:7.4294，年化波动率为：0.45
股票编号3442，第5年份年末最后20个交易日收盘价加权平均值为:5.8498，年化波动率为：0.72
股票编号3442，第6年份年末最后20个交易日收盘价加权平均值为:4.1334，年化波动率为：0.35
股票编号3442，第7年份年末最后20个交易日收盘价加权平均值为:3.4727，年化波动率为：0.26
股票编号

股票编号3464，第5年份年末最后20个交易日收盘价加权平均值为:4.6547，年化波动率为：0.61
股票编号3464，第6年份年末最后20个交易日收盘价加权平均值为:5.8694，年化波动率为：0.49
股票编号3464，第7年份年末最后20个交易日收盘价加权平均值为:8.9758，年化波动率为：0.51
股票编号3465，第1年份年末最后20个交易日收盘价加权平均值为:2.9278，年化波动率为：0.23
股票编号3465，第2年份年末最后20个交易日收盘价加权平均值为:3.2698，年化波动率为：0.22
股票编号3465，第3年份年末最后20个交易日收盘价加权平均值为:3.0511，年化波动率为：0.31
股票编号3465，第4年份年末最后20个交易日收盘价加权平均值为:4.8978，年化波动率为：0.30
股票编号3465，第5年份年末最后20个交易日收盘价加权平均值为:6.2158，年化波动率为：0.48
股票编号3465，第6年份年末最后20个交易日收盘价加权平均值为:7.3659，年化波动率为：0.19
股票编号3465，第7年份年末最后20个交易日收盘价加权平均值为:6.6083，年化波动率为：0.14
股票编号3466，第1年份年末最后20个交易日收盘价加权平均值为:6.0555，年化波动率为：0.00
股票编号3466，第2年份年末最后20个交易日收盘价加权平均值为:6.0555，年化波动率为：0.00
股票编号3466，第3年份年末最后20个交易日收盘价加权平均值为:6.0555，年化波动率为：0.00
股票编号3466，第4年份年末最后20个交易日收盘价加权平均值为:19.6823，年化波动率为：0.45
股票编号3466，第5年份年末最后20个交易日收盘价加权平均值为:25.5539，年化波动率为：0.80
股票编号3466，第6年份年末最后20个交易日收盘价加权平均值为:26.6287，年化波动率为：0.51
股票编号3466，第7年份年末最后20个交易日收盘价加权平均值为:21.1234，年化波动率为：0.32


In [14]:
day_fac = keys.copy()
day_fac['股价（年末最后20个交易日收盘价加权平均）'] = price_lists
day_fac['股价年化波动率'] = vol_lists
day_fac.head()

,股票编号,年份（年末）,股价（年末最后20个交易日收盘价加权平均）,股价年化波动率
0,1,1,6.186360,0.500901
1,1,2,5.812830,0.490998
2,1,3,7.159914,0.423839
3,1,4,9.088111,0.342857
4,1,5,13.947512,0.754430


In [16]:
# 保存数据
day_fac.to_csv('temp/股价均价和波动率.csv',index = False, encoding = 'GBK')